In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Optional, List, Tuple

from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        print(output)

    return Printer

@legoPrinter
@beartype
def k8s_list_pvcs(handle, namespace: str = '') -> List:
    if namespace == '':
        kubectl_command = 'kubectl get pvc -A --output=jsonpath=\'{range .items[*]}{@.metadata.namespace}{","}{@.metadata.name}{"\\n"}{end}\''
    else:
        kubectl_command = 'kubectl get pvc -n ' + namespace + ' --output=jsonpath=\'{range .items[*]}{@.metadata.namespace}{","}{@.metadata.name}{"\\n"}{end}\''
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None
    names_list = [y for y in (x.strip() for x in result.stdout.splitlines()) if y]
    output = []
    for i in names_list:
        ns, name = i.split(",")
        output.append({"Namespace": ns, "Name":name})
    return output


task = Task(Workflow())
task.configure(outputName="pvcsList")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_list_pvcs, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

In [20]:
Value = float(Value)

In [21]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from unskript.connectors.infra import InfraConnector
from typing import Optional
import requests
from polling2 import poll_decorator
import html_to_json
import uuid 

class Schema(BaseModel):
    Namespace: Optional[str] = Field(
        None, description='Namespace of the PVC', title='Namespace'
    )
    PVCName: Optional[str] = Field(None, description='Name of the PVC', title='PVCName')
    ResizeOption: Optional[str] = Field(
        'Add',
        description='Option to resize the volume. 2 options supported:             1. Add - Use this option to resize by an amount.             2. Multiple - Use this option if you want to resize by a multiple of the current volume size.',
        title='ResizeOption',
    )
    RestartPodsAfterResize: Optional[bool] = Field(
        False,
        description='Restart the pods after PVC resize',
        title='RestartPodsAfterResize',
    )
    Channel: Optional[str] = Field(
        None,
        description='Slack Channel name where notification will be send.',
        title='SlackChannelName',
    )
    Value: Optional[float] = Field(
        None,
        description='Based on the resize option chosen, specify the value. For eg, if you chose Add option, this             value will be a value in Gb (like 100). If you chose, this value will be a multiplying factor             to the current volume size. For eg, to double, specify value as 2.',
        title='Value',
    )

@poll_decorator(step=10, timeout=60, check_success=lambda x: x is True)
def checkExecutionStatus(handle, tenantID, executionID) -> bool:
    print(f'Checking execution status')
    url = f'{env["TENANT_URL"]}/executions/{executionID}'
    try:
        resp = handle.request('GET', url, params={'tenant_id': tenantID, "summary": True})
        resp.raise_for_status()
    except Exception as e:
        print(f'Get execution {executionID} failed, {e}')
        return False

    try:
        result = resp.json()
    except Exception:
        result = html_to_json.convert(resp.content)
    if result['execution']['executionStatus'] == "EXECUTION_STATUS_SUCCEEDED" or result['execution']['executionStatus'] == "EXECUTION_STATUS_FAILED":
        return True
    else:
        return False


from beartype import beartype
@beartype
def call_pvc_resize_runbook(handle: InfraConnector, Namespace: str, PVCName: str, ResizeOption: str, RestartPodsAfterResize:bool, Value: float, Channel: str = None):
    workflowIDToBeCalled = RunbookID
    apiToken = APIToken
    tenantID = env['TENANT_ID']
    environmentID = env['ENVIRONMENT_ID']
    userID = "Bot-user"

    params = Schema()
    params.Namespace = Namespace
    params.PVCName = PVCName
    params.Value = Value
    params.ResizeOption = ResizeOption
    params.Channel = Channel
    payload = {
        "req_hdr": {
            "tid": str(uuid.uuid4())
        },
        "tenant_id": tenantID,
        "environment_id": environmentID,
        "user_id": userID,
        "params": params.json()
    }
    handle = requests.Session()
    authHeader = f'unskript-sha {apiToken}'
    handle.headers.update({'Authorization': authHeader})
    url = f'{env["TENANT_URL"]}/workflows/{workflowIDToBeCalled}/run'

    try:
        resp = handle.request('POST', url, json=payload)
        resp.raise_for_status()
    except Exception as e:
        print(f'Workflow run failed, {e}')
        raise e

    try:
        result = resp.json()
    except Exception:
        result = html_to_json.convert(resp.content)

    executionID = result['executionId']
    print(f'ExecutionID {executionID}')

    try:
        checkExecutionStatus(handle, tenantID, executionID)
    except Exception as e:
        handle.close()
        print(f'Check execution status for {executionID} failed, {e}')
        raise e

    handle.close()
    return

task = Task(Workflow())
task.configure(inputParamsJson='''{
    "Namespace": {
        "constant": false,
        "value": "iter.get(\\\"Namespace\\\")"
    },
    "PVCName": {
        "constant": false,
        "value": "iter.get(\\\"Name\\\")"
    },
    "ResizeOption": {
        "constant": false,
        "value": "ResizeOption"
    },
    "RestartPodsAfterResize": {
        "constant": true,
        "value": false
    },
    "Channel": {
        "constant": false,
        "value": "Channel"
    },
    "Value": {
        "constant": false,
        "value": "Value"
    }
}''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "pvcsList",
    "iter_parameter": [
        "Namespace",
        "PVCName"
    ]
}''')



(err, hdl, args) = task.validate(vars=vars(), infra=True)
if err is None:
    task.output = task.execute(call_pvc_resize_runbook, hdl, args)
if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(task.output)